# MS-Swift Qwen 模型推理示例

本 notebook 演示如何使用 MS-Swift 框架对 Qwen 模型进行推理，解决常见的兼容性问题，并展示正确的 API 使用方法。

## 目标
1. 正确配置 MS-Swift 推理环境
2. 解决 PyTorch 兼容性问题
3. 演示正确的推理 API 用法
4. 展示输出结果的正确处理方式

## 1. 环境设置和依赖检查

首先检查和配置推理环境，确保所有依赖都正确安装。

In [4]:
# 检查基础环境
import sys
import torch
import os

print(f"Python 版本: {sys.version}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA 版本: {torch.version.cuda}")
    print(f"GPU 数量: {torch.cuda.device_count()}")
    print(f"当前 GPU: {torch.cuda.current_device()}")

# 设置 GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
print(f"设置使用 GPU: {os.environ['CUDA_VISIBLE_DEVICES']}")

Python 版本: 3.10.12 (main, Aug 15 2025, 14:32:43) [GCC 11.4.0]
PyTorch 版本: 2.8.0+cu128
CUDA 可用: True
CUDA 版本: 12.8
GPU 数量: 1
当前 GPU: 0
设置使用 GPU: 0


In [2]:
# 修复环境配置问题
import os

# 清除可能导致问题的环境变量
if 'BNB_CUDA_VERSION' in os.environ:
    print(f"清除 BNB_CUDA_VERSION: {os.environ['BNB_CUDA_VERSION']}")
    del os.environ['BNB_CUDA_VERSION']

# 设置正确的库路径
cuda_lib_path = "/usr/local/cuda/lib64"
if cuda_lib_path not in os.environ.get('LD_LIBRARY_PATH', ''):
    os.environ['LD_LIBRARY_PATH'] = f"{cuda_lib_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"
    print(f"设置 LD_LIBRARY_PATH: {os.environ['LD_LIBRARY_PATH']}")

print("环境配置修复完成")

清除 BNB_CUDA_VERSION: 123
设置 LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/lib/python3.10/dist-packages/torch/lib:/usr/local/lib/python3.10/dist-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda-12.6:/usr/local/cuda-12.6/include:/usr/include/x86_64-linux-gnu
环境配置修复完成


In [3]:
# 测试 bitsandbytes 是否正常工作
try:
    import bitsandbytes as bnb
    print(f"✅ bitsandbytes 版本: {bnb.__version__}")
except Exception as e:
    print(f"❌ bitsandbytes 导入失败: {e}")
    
# 测试 MS-Swift 导入
try:
    from swift.llm import PtEngine, RequestConfig, InferRequest
    print("✅ MS-Swift 核心模块导入成功")
except Exception as e:
    print(f"❌ MS-Swift 导入失败: {e}")
    print("如果遇到 scaled_dot_product_attention 错误，这是已知的兼容性问题")

✅ bitsandbytes 版本: 0.48.1


/home/work/hd/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ MS-Swift 核心模块导入成功


## 2. 兼容性问题解决方案

当前环境中存在 PyTorch nightly 版本与 transformers 库的兼容性问题。我们提供几种解决方案：

In [ ]:
# 方案A: 使用 transformers 直接推理（推荐）
print("=== 方案A: 使用 transformers 直接推理 ===")
print("这是最稳定的解决方案，绕过了 MS-Swift 的兼容性问题")

try:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    print("✅ transformers 库可用")
    
    # 检查模型路径
    model_path = "/home/work/hd/models/Qwen3-4B-Thinking-2507-FP8"
    import os
    if os.path.exists(model_path):
        print(f"✅ 模型路径存在: {model_path}")
    else:
        print(f"❌ 模型路径不存在: {model_path}")
        
except Exception as e:
    print(f"❌ transformers 导入失败: {e}")

## 3. 使用 transformers 进行推理

让我们使用 transformers 库直接进行推理，这避免了当前的兼容性问题：

In [ ]:
# 使用 transformers 加载模型和分词器
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_path = "/home/work/hd/models/Qwen3-4B-Thinking-2507-FP8"

print("正在加载模型...")
try:
    # 加载分词器
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print("✅ 分词器加载成功")
    
    # 加载模型（使用 torch.bfloat16 以节省显存）
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True
    )
    print("✅ 模型加载成功")
    
    print(f"模型设备: {model.device}")
    print(f"模型参数量: {sum(p.numel() for p in model.parameters())/1e9:.2f}B")
    
except Exception as e:
    print(f"❌ 模型加载失败: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 推理测试
def chat_with_model(prompt, max_tokens=512, temperature=0.7):
    """使用模型进行对话"""
    try:
        # 构建对话格式
        messages = [
            {"role": "user", "content": prompt}
        ]
        
        # 应用聊天模板
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        
        # 编码输入
        inputs = tokenizer(text, return_tensors="pt").to(model.device)
        
        # 生成回复
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                temperature=temperature,
                do_sample=True if temperature > 0 else False,
                pad_token_id=tokenizer.eos_token_id
            )
        
        # 解码输出
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # 提取新生成的部分
        response = response[len(text):].strip()
        
        return response
        
    except Exception as e:
        return f"推理失败: {e}"

# 测试推理
test_prompt = "请详细解释一下大语言模型的"思考模式（Thinking mode）"是什么？"
print(f"用户: {test_prompt}")
print("=" * 50)
print("AI: ", end="", flush=True)

response = chat_with_model(test_prompt, max_tokens=512, temperature=0.7)
print(response)

## 4. MS-Swift API 正确用法（待兼容性修复后使用）

以下展示了 MS-Swift 的正确 API 用法，基于官方示例：

In [ ]:
# MS-Swift 正确的 API 用法（示例代码）
# 注意：当前环境下会因为兼容性问题而失败

from swift.llm import PtEngine, RequestConfig, InferRequest

def swift_inference_example():
    """
    MS-Swift 推理的正确用法
    基于官方 GitHub 示例：https://github.com/modelscope/ms-swift
    """
    # 1. 创建推理引擎
    model_path = "/home/work/hd/models/Qwen3-4B-Thinking-2507-FP8"
    
    engine = PtEngine(
        model_path,
        max_batch_size=1,
        dtype='fp8',
        # 可以尝试添加这些参数来解决兼容性问题
        # attn_implementation='eager',
        # torch_dtype=torch.bfloat16
    )
    
    # 2. 创建请求配置
    request_config = RequestConfig(
        max_tokens=512,
        temperature=0.7,
        top_p=0.9
    )
    
    # 3. 创建推理请求
    infer_request = InferRequest(
        messages=[
            {"role": "user", "content": "请详细解释一下大语言模型的"思考模式（Thinking mode）"是什么？"}
        ]
    )
    
    # 4. 执行推理
    outputs = engine.infer([infer_request], request_config)
    
    # 5. 正确的输出访问方式（重要！）
    response = outputs[0].choices[0].message.content
    
    return response

# 展示正确的输出结构
print("MS-Swift 推理的正确输出访问方式：")
print("outputs = engine.infer([infer_request], request_config)")
print("response = outputs[0].choices[0].message.content")
print()
print("注意：这与 OpenAI API 的结构保持一致")

## 5. 修复原始脚本

让我们修复您的原始 `1.py` 脚本中的问题：

In [ ]:
# 原始脚本的问题和修复方案

print("=== 原始脚本存在的问题 ===")
print("1. 模型路径错误")
print("   错误: /home/work/models/Qwen3-4B-Thinking-2507-FP8")
print("   正确: /home/work/hd/models/Qwen3-4B-Thinking-2507-FP8")
print()

print("2. 兼容性问题")
print("   PyTorch nightly 版本与 transformers 不兼容")
print("   错误: scaled_dot_product_attention() got unexpected keyword argument 'enable_gqa'")
print()

print("3. 输出访问方式（这个实际是正确的）")
print("   正确: outputs[0].choices[0].message.content")
print()

# 创建修复后的脚本内容
fixed_script = '''import os
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 指定GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# 修复后的模型路径
model_path = "/home/work/hd/models/Qwen3-4B-Thinking-2507-FP8"

print("正在加载模型...")

# 使用 transformers 直接加载（避免兼容性问题）
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# 推理函数
def generate_response(prompt, max_tokens=512, temperature=0.7):
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(text):].strip()

# 执行推理
prompt = "请详细解释一下大语言模型的"思考模式（Thinking mode）"是什么？"
response = generate_response(prompt)
print("Response:", response)
'''

# 保存修复后的脚本
with open('/home/work/hd/1_fixed.py', 'w', encoding='utf-8') as f:
    f.write(fixed_script)
    
print("✅ 修复后的脚本已保存为 1_fixed.py")

## 6. 解决方案总结

### 当前推荐的解决方案：

1. **使用 transformers 直接推理**（最稳定）
   - 避免了当前的兼容性问题
   - 性能和功能基本一致
   - 代码更简洁，更易理解

2. **MS-Swift API 的正确用法**（待环境修复后使用）
   - `outputs = engine.infer([infer_request], request_config)`
   - `response = outputs[0].choices[0].message.content`

### 兼容性问题的根本原因：
- PyTorch nightly 版本 (2.5.0a0) 
- transformers 库中的 `scaled_dot_product_attention` 函数参数变化
- `enable_gqa` 参数在新版本中被移除

### 长期解决方案：
1. 降级到稳定版本的 PyTorch
2. 更新 transformers 到兼容版本  
3. 等待 MS-Swift 更新以支持新版本